# Code  
---
This code is taken from: https://github.com/rafikg/CEAL
## Import all the required stuff

In [1]:
# imported from run_ceal/ceal_learning_algorithm.py
from model import AlexNet
from utils import Normalize, RandomCrop, SquarifyImage, \
    ToTensor, GameImageDataset
from utils import get_uncertain_samples, get_high_confidence_samples, \
    update_threshold
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import torch
import logging


# others
from sklearn.metrics import precision_recall_fscore_support as prfs

logging.basicConfig(format="%(levelname)s:%(name)s: %(message)s",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
device = torch.device('cuda:1')

## CEAL Algorithm

In [3]:
def ceal_learning_algorithm(du: DataLoader,
                            dl: DataLoader,
                            dtest: DataLoader,
                            k: int = 20,
                            delta_0: float = 0.005,
                            dr: float = 0.00033,
                            t: int = 1,
                            epochs: int = 10,
                            criteria: str = 'cl',
                            max_iter: int = 15):
    """
    Algorithm1 : Learning algorithm of CEAL.
    For simplicity, I used the same notation in the paper.
    Parameters
    ----------
    du: DataLoader
        Unlabeled samples
    dl : DataLoader
        labeled samples
    dtest : DataLoader
        test data
    k: int, (default = 1000)
        uncertain samples selection
    delta_0: float
        hight confidence samples selection threshold
    dr: float
        threshold decay
    t: int
        fine-tuning interval
    epochs: int
    criteria: str
    max_iter: int
        maximum iteration number.

    Returns
    -------

    """
    logger.info('Initial configuration: len(du): {}, len(dl): {} '.format(
        len(du.sampler.indices),
        len(dl.sampler.indices)))

    # Create the model
    model = AlexNet(n_classes=4, device=device)

    # Initialize the model
    logger.info('Intialize training the model on `dl` and test on `dtest`')

    model.train(epochs=epochs, train_loader=dl, valid_loader=None)

    # Evaluate model on dtest
    p, r, f = model.evaluate(test_loader=dtest)

    print('====> Initial precision: {} '.format(sum(p)/4))
    print('====> Initial recall: {} '.format(sum(r)/4))
    print('====> Initial f-score: {} '.format(sum(f)/4))

    for iteration in range(max_iter):

        logger.info('Iteration: {}: run prediction on unlabeled data '
                    '`du` '.format(iteration))

        pred_prob = model.predict(test_loader=du)

        # get k uncertain samples
        uncert_samp_idx, _ = get_uncertain_samples(pred_prob=pred_prob, k=k,
                                                   criteria=criteria)

        # get original indices
        uncert_samp_idx = [du.sampler.indices[idx] for idx in uncert_samp_idx]

        # add the uncertain samples selected from `du` to the labeled samples
        #  set `dl`
        dl.sampler.indices.extend(uncert_samp_idx)

        logger.info(
            'Update size of `dl`  and `du` by adding uncertain {} samples'
            ' in `dl`'
            ' len(dl): {}, len(du) {}'.
            format(len(uncert_samp_idx), len(dl.sampler.indices),
                   len(du.sampler.indices)))

        # get high confidence samples `dh`
        hcs_idx, hcs_labels = get_high_confidence_samples(pred_prob=pred_prob,
                                                          delta=delta_0)
        # get the original indices
        hcs_idx = [du.sampler.indices[idx] for idx in hcs_idx]

        # remove the samples that already selected as uncertain samples.
        hcs_idx = [x for x in hcs_idx if
                   x not in list(set(uncert_samp_idx) & set(hcs_idx))]

        # add high confidence samples to the labeled set 'dl'

        # (1) update the indices
        dl.sampler.indices.extend(hcs_idx)
        # (2) update the original labels with the pseudo labels.
        for idx in range(len(hcs_idx)):
            x = [0, 0, 0, 0]
            x[hcs_labels[idx]] = 1    
            dl.dataset.labels[hcs_idx[idx]] = x
        logger.info(
            'Update size of `dl`  and `du` by adding {} hcs samples in `dl`'
            ' len(dl): {}, len(du) {}'.
            format(len(hcs_idx), len(dl.sampler.indices),
                   len(du.sampler.indices)))

        if iteration % t == 0:
            logger.info('Iteration: {} fine-tune the model on dh U dl'.
                        format(iteration))
            model.train(epochs=epochs, train_loader=dl)

            # update delta_0
            delta_0 = update_threshold(delta=delta_0, dr=dr, t=iteration)

        # remove the uncertain samples from the original `du`
        logger.info('remove {} uncertain samples from du'.
                    format(len(uncert_samp_idx)))
        for val in uncert_samp_idx:
            du.sampler.indices.remove(val)

        p, r, f = model.evaluate(test_loader=dtest)
                
        print(
            "Iteration: {}, len(dl): {}, len(du): {},"
            " len(dh) {}".format(
                iteration, len(dl.sampler.indices),
                len(du.sampler.indices), len(hcs_idx)))
        
        print("Precision:",sum(p)/4)
        print("Recall:",sum(r)/4)
        print("F1:",sum(f)/4)
        
    print("Saving model...")
    torch.save(model, 'saved_model/alexnet.pth')
    print("Model saved as: saved_model/alexnet.pth")

In [4]:
dataset_train = GameImageDataset(
    root_dir="data/train",
    transform=transforms.Compose(
        [SquarifyImage(),
         RandomCrop(224),
         Normalize(),
         ToTensor()]))

dataset_test = GameImageDataset(
    root_dir="data/test",
    transform=transforms.Compose(
        [SquarifyImage(),
         RandomCrop(224),
         Normalize(),
         ToTensor()]))

# Creating data indices for training and validation splits:
random_seed = 123
validation_split = 0.1  # 10%
shuffling_dataset = True
batch_size = 16
dataset_size = len(dataset_train)

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffling_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

du = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                 sampler=train_sampler, num_workers=1)
dl = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                 sampler=valid_sampler, num_workers=1)
dtest = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
                                    num_workers=1)

ceal_learning_algorithm(du=du, dl=dl, dtest=dtest, k=60, max_iter=5, epochs=30)

INFO:__main__: Initial configuration: len(du): 314, len(dl): 34 
INFO:model.alexnet: The code is running on cuda:1 
INFO:__main__: Intialize training the model on `dl` and test on `dtest`


Train Epoch: 0 [0/34 (0%)]	Loss: 0.714026
====> Epoch: 0 Average loss: 0.0707
Train Epoch: 1 [0/34 (0%)]	Loss: 0.594891
====> Epoch: 1 Average loss: 0.0521
Train Epoch: 2 [0/34 (0%)]	Loss: 0.678528
====> Epoch: 2 Average loss: 0.0459
Train Epoch: 3 [0/34 (0%)]	Loss: 0.479437
====> Epoch: 3 Average loss: 0.0501
Train Epoch: 4 [0/34 (0%)]	Loss: 0.461799
====> Epoch: 4 Average loss: 0.0332
Train Epoch: 5 [0/34 (0%)]	Loss: 0.373035
====> Epoch: 5 Average loss: 0.0406
Train Epoch: 6 [0/34 (0%)]	Loss: 0.373514
====> Epoch: 6 Average loss: 0.0265
Train Epoch: 7 [0/34 (0%)]	Loss: 0.291382
====> Epoch: 7 Average loss: 0.0259
Train Epoch: 8 [0/34 (0%)]	Loss: 0.383160
====> Epoch: 8 Average loss: 0.0249
Train Epoch: 9 [0/34 (0%)]	Loss: 0.196360
====> Epoch: 9 Average loss: 0.0250
Train Epoch: 10 [0/34 (0%)]	Loss: 0.265015
====> Epoch: 10 Average loss: 0.0189
Train Epoch: 11 [0/34 (0%)]	Loss: 0.152182
====> Epoch: 11 Average loss: 0.0132
Train Epoch: 12 [0/34 (0%)]	Loss: 0.123265
====> Epoch: 12 A

INFO:__main__: Iteration: 0: run prediction on unlabeled data `du` 


====> Initial precision: 0.7089039432789431 
====> Initial recall: 0.7070942945942946 
====> Initial f-score: 0.6880104379257193 


INFO:__main__: Update size of `dl`  and `du` by adding uncertain 60 samples in `dl` len(dl): 94, len(du) 314
INFO:__main__: Update size of `dl`  and `du` by adding 0 hcs samples in `dl` len(dl): 94, len(du) 314
INFO:__main__: Iteration: 0 fine-tune the model on dh U dl


...

Iteration: 3, len(dl): 274, len(du): 74, len(dh) 0
Precision: 0.6859481375106375
Recall: 0.6854937597125097
F1: 0.6669745944034061


INFO:__main__: Update size of `dl`  and `du` by adding uncertain 60 samples in `dl` len(dl): 334, len(du) 74
INFO:__main__: Update size of `dl`  and `du` by adding 0 hcs samples in `dl` len(dl): 334, len(du) 74
INFO:__main__: Iteration: 4 fine-tune the model on dh U dl


Train Epoch: 0 [0/334 (0%)]	Loss: 0.333235
====> Epoch: 0 Average loss: 0.0169
Train Epoch: 1 [0/334 (0%)]	Loss: 0.249951
====> Epoch: 1 Average loss: 0.0159
Train Epoch: 2 [0/334 (0%)]	Loss: 0.233464
====> Epoch: 2 Average loss: 0.0166
Train Epoch: 3 [0/334 (0%)]	Loss: 0.160521
====> Epoch: 3 Average loss: 0.0143
Train Epoch: 4 [0/334 (0%)]	Loss: 0.373476
====> Epoch: 4 Average loss: 0.0153
Train Epoch: 5 [0/334 (0%)]	Loss: 0.230807
====> Epoch: 5 Average loss: 0.0149
Train Epoch: 6 [0/334 (0%)]	Loss: 0.146505
====> Epoch: 6 Average loss: 0.0138
Train Epoch: 7 [0/334 (0%)]	Loss: 0.177746
====> Epoch: 7 Average loss: 0.0145
Train Epoch: 8 [0/334 (0%)]	Loss: 0.221866
====> Epoch: 8 Average loss: 0.0136
Train Epoch: 9 [0/334 (0%)]	Loss: 0.183206
====> Epoch: 9 Average loss: 0.0131
Train Epoch: 10 [0/334 (0%)]	Loss: 0.376792
====> Epoch: 10 Average loss: 0.0124
Train Epoch: 11 [0/334 (0%)]	Loss: 0.188778
====> Epoch: 11 Average loss: 0.0140
Train Epoch: 12 [0/334 (0%)]	Loss: 0.118978
====

INFO:__main__: remove 60 uncertain samples from du


====> Epoch: 29 Average loss: 0.0121
Iteration: 4, len(dl): 334, len(du): 14, len(dh) 0
Precision: 0.7328382554945055
Recall: 0.7235808173308174
F1: 0.7097713661123549
Saving model...
Model saved as: saved_model/alexnet.pth


# Extra processing
---
#### The cells are converted to markdown cells so that they are not run when you run all cells. Change them to code cells to run if required

### Exporting data zip file

import zipfile
with zipfile.ZipFile('data/annotated_r2.zip', 'r') as zip_ref:
    zip_ref.extractall()

### Move files to train and test directory

import os
dataitems = os.listdir('data')
finalitems = []
for item in dataitems:
    if not os.path.isdir('data/'+item):
        finalitems.append(item)

for idx, item in enumerate(finalitems):
    # this will assign 20% images(every 5th image) to the test directory
    if idx % 5 == 0: 
        os.rename('data/' + item, 'data/test/'+item)
    else:
        os.rename('data/' + item, 'data/train/'+item)

### Create small train-test set for debugging

import os
from shutil import copyfile

trainitems = os.listdir('data/train')
testitems = os.listdir('data/test')

for idx, item in enumerate(trainitems):
    if os.path.isdir('data/train/'+item):
        continue
        
    # this will assign 20% images(every 5th image) to the debug directory
    if idx % 5 == 0: 
        copyfile('data/train/'+item,'data/debug_train/'+item)
        
for idx, item in enumerate(testitems):
    if os.path.isdir('data/train/'+item):
        continue
        
    # this will assign 20% images(every 5th image) to the debug directory
    if idx % 5 == 0: 
        copyfile('data/test/'+item,'data/debug_test/'+item)